# Inception V3 Architecture

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
def inception_module(x, f1, f2, f3):
	conv1 =  keras.layers.Conv2D(f1, (1,1), padding='same', activation='relu')(x)
	conv3 = keras.layers.Conv2D(f2, (3,3), padding='same', activation='relu')(x)
	conv5 = keras.layers.Conv2D(f3, (5,5), padding='same', activation='relu')(x)
	pool = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
	out = keras.layers.merge.concatenate([conv1, conv3, conv5, pool])
	return out

In [ ]:
def conv2d_bn(x,filters,num_row,num_col,padding='same',strides=(1, 1)):
    x = keras.layers.Conv2D(filters, (num_row, num_col),strides=strides,padding=padding)(x)
    x = keras.layers.BatchNormalization(axis=3, scale=False)(x)
    x = keras.layers.Activation('relu')(x)
    return x

In [ ]:
channel_axis = -1
def inc_block_A(x):
    branch_1 = conv2d_bn(x, 64, 1, 1)
    branch_5 = conv2d_bn(x, 48, 1, 1)
    branch_5 = conv2d_bn(branch_5, 64, 5, 5)

    branch_3 = conv2d_bn(x, 64, 1, 1)
    branch_3 = conv2d_bn(branch_3, 96, 3, 3)
    branch_3 = conv2d_bn(branch_3, 96, 3, 3)

    branch_pooling = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pooling = conv2d_bn(branch_pooling, 32, 1, 1)
    x = keras.layers.concatenate([branch_1, branch_5, branch_3, branch_pooling], axis=channel_axis)
    return x

def reduction_block_a(x):
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = keras.layers.concatenate([branch3x3, branch3x3dbl, branch_pool],axis=channel_axis)
    return x

In [ ]:
def inc_block_b(x):
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1),padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = keras.layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool], axis=channel_axis)
    return x

def reduction_block_b(x):
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,strides=(2, 2), padding='valid')

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn( branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = keras.layers.concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis)
    return x

In [ ]:
def inc_block_c(x):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = keras.layers.concatenate([branch3x3_1, branch3x3_2],axis=channel_axis)

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = keras.layers.concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

        branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = keras.layers.concatenate( [branch1x1, branch3x3, branch3x3dbl, branch_pool],axis=channel_axis)
        return x

In [ ]:
img_input = keras.Input(shape=(299, 299, 3))

In [ ]:
img_input = keras.Input(shape=(299, 299, 3))
x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
x = conv2d_bn(x, 32, 3, 3, padding='valid')
x = conv2d_bn(x, 64, 3, 3)

x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
x = conv2d_bn(x, 80, 1, 1, padding='valid')
x = conv2d_bn(x, 192, 3, 3, padding='valid')
x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)


x=inc_block_A(x)
x=inc_block_A(x)
x=inc_block_A(x)

x=reduction_block_a(x)

x=inc_block_b(x)
x=inc_block_b(x)
x=inc_block_b(x)
x=inc_block_b(x)
x=reduction_block_b(x)
x=inc_block_c(x)
x=inc_block_c(x)

x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
classes =1000
x = keras.layers.Dense(units= classes, activation='softmax', name='predictions')(x)

In [ ]:
inputs = img_input
model =  keras.Model(inputs, x, name='inception_v3')
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_535 (Conv2D)         (None, 149, 149, 32)         896       ['input_13[0][0]']            
                                                                                                  
 batch_normalization_507 (B  (None, 149, 149, 32)         96        ['conv2d_535[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_405 (Activation  (None, 149, 149, 32)         0         ['batch_normalizati

importing Inception V3 from pre trained model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
model=InceptionV3()
print(model.summary())

96112376/96112376 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_629 (Conv2D)         (None, 149, 149, 32)         864       ['input_14[0][0]']            
                                                                                                  
 batch_normalization_601 (B  (None, 149, 149, 32)         96        ['conv2d_629[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_499 (Activ